In [17]:
import psycopg2

In [22]:
import logging as logger

In [28]:
import pandas as pd

In [6]:
from sqlalchemy import create_engine

In [37]:
import psycopg2
import pandas as pd

# Connection parameters, yours will be different
param_dic = {
    "host"      : "localhost",
    "database"  : "moodle",
    "user"      : "tenac",
    "password"  : "password"
}


In [42]:

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn,select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    #db = DBHandler()
    #db.open_local_db()
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1

    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()

    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df



In [47]:
conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [50]:
column_names = ["Final grade", "User Id"]
# Execute the "SELECT *" query
df = postgresql_to_dataframe(conn, "select finalgrade, userid from mdl_grade_grades", column_names)
df.head()

,Final grade,User Id
0,None,3
1,None,3
2,12.00000,50
3,0.00000,65
4,0.00000,65
